<a href="https://colab.research.google.com/github/sbbaik/small_bert/blob/main/small_bert_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 드라이브에 저장할 경로 설정 (미리 폴더를 생성해두는 것이 좋습니다)
SAVE_PATH = '/content/drive/MyDrive/small_bert_korean'

Mounted at /content/drive


In [2]:
!pip install tokenizers

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tokenizers import BertWordPieceTokenizer
import os
import math

In [8]:
import torch
import torch.nn as nn

# 하이퍼파라미터 정의
d_model = 768  # 일반 BERT-base와 동일한 임베딩 차원
n_head = 12    # 일반 BERT-base와 동일한 헤드 수
num_layers = 6 # BERT-base의 절반
dim_feedforward = d_model * 4
max_len = 50
vocab_size = 30000  # 예시로 설정한 어휘 크기
dropout = 0.1

In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=50):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [13]:
# 수정된 SmallBERT 클래스
class SmallBERT(nn.Module):
    def __init__(self, vocab_size, d_model, n_head, num_layers, dim_feedforward, max_len, dropout):
        super(SmallBERT, self).__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_head,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        # 새로운 선형 레이어 추가: d_model -> vocab_size로 차원을 변환
        self.output_layer = nn.Linear(d_model, vocab_size)

    def forward(self, src):
        # src: (batch_size, seq_len)
        x = self.embedding(src) * math.sqrt(d_model)
        x = self.pos_encoder(x)
        output = self.transformer_encoder(x)

        # 마지막에 output_layer를 통과시켜 vocab_size 차원으로 변환
        output = self.output_layer(output)

        return output

In [11]:
model = SmallBERT(
    vocab_size=vocab_size,
    d_model=d_model,
    n_head=n_head,
    num_layers=num_layers,
    dim_feedforward=dim_feedforward,
    max_len=max_len,
    dropout=dropout
)

print(model)

SmallBERT(
  (embedding): Embedding(30000, 768)
  (pos_encoder): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (linear1): Linear(in_features=768, out_features=3072, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=3072, out_features=768, bias=True)
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
)


In [4]:
import os
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 저장할 경로 설정
SAVE_PATH = '/content/drive/MyDrive/small_bert_korean'

# 디렉토리가 없으면 생성
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
    print(f"디렉토리 '{SAVE_PATH}'가 생성되었습니다.")

# 이제 파일을 저장하는 코드를 실행
text_data = "안녕하세요. 저는 작은 bert 모델을 만들고 있습니다. 자연어 처리는 정말 재미있습니다. 한국어 데이터로 사전 학습을 진행합니다." * 1000
with open(os.path.join(SAVE_PATH, 'korean_text.txt'), 'w', encoding='utf-8') as f:
    f.write(text_data)

print("파일이 성공적으로 생성되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
디렉토리 '/content/drive/MyDrive/small_bert_korean'가 생성되었습니다.
파일이 성공적으로 생성되었습니다.


In [5]:
# 예시 텍스트 파일 생성
# 실제로는 대용량 한국어 텍스트 파일을 사용해야 합니다.
text_data = "안녕하세요. 저는 작은 bert 모델을 만들고 있습니다. 자연어 처리는 정말 재미있습니다. 한국어 데이터로 사전 학습을 진행합니다." * 1000
with open(os.path.join(SAVE_PATH, 'korean_text.txt'), 'w', encoding='utf-8') as f:
    f.write(text_data)

# 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size=50):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.examples = []
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        tokenized_text = self.tokenizer.encode(text)
        token_ids = tokenized_text.ids

        # 텍스트를 고정된 길이의 블록으로 분할
        for i in range(0, len(token_ids) - block_size + 1, block_size):
            self.examples.append(torch.tensor(token_ids[i:i+block_size], dtype=torch.long))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [6]:
# 토크나이저 학습 및 저장
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=False
)

files = [os.path.join(SAVE_PATH, 'korean_text.txt')]

tokenizer.train(
    files,
    vocab_size=30000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
)

# 학습된 토크나이저를 드라이브에 저장
tokenizer.save_model(SAVE_PATH)

['/content/drive/MyDrive/small_bert_korean/vocab.txt']

In [14]:
# 모델 하이퍼파라미터 정의
d_model = 768
n_head = 12
num_layers = 6
dim_feedforward = d_model * 4
max_len = 50
dropout = 0.1
vocab_size = tokenizer.get_vocab_size()

# 모델 클래스 (이전 응답에서 정의한 내용과 동일)
# PositionalEncoding 및 SmallBERT 클래스 코드를 여기에 다시 붙여넣기

# 모델 인스턴스 생성
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SmallBERT(vocab_size, d_model, n_head, num_layers, dim_feedforward, max_len, dropout).to(device)

# 데이터로더 설정
dataset = TextDataset(os.path.join(SAVE_PATH, 'korean_text.txt'), tokenizer, block_size=max_len)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 옵티마이저 및 손실 함수
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# 사전 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        inputs = batch.to(device)

        # MLM을 위한 마스킹
        labels = inputs.clone().detach()
        masked_indices = torch.rand(inputs.shape) < 0.15
        inputs[masked_indices] = tokenizer.token_to_id("[MASK]")

        optimizer.zero_grad()
        outputs = model(inputs)

        # 마스킹된 토큰에 대한 손실 계산
        loss = criterion(outputs.view(-1, vocab_size), labels.view(-1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Perplexity: {math.exp(avg_loss):.4f}")

    # 중간 체크포인트 저장 (구글 드라이브)
    checkpoint_path = os.path.join(SAVE_PATH, f'model_checkpoint_epoch_{epoch+1}.pt')
    torch.save(model.state_dict(), checkpoint_path)
    print(f"모델 체크포인트가 {checkpoint_path}에 저장되었습니다.")

Epoch [1/10], Loss: 1.3719, Perplexity: 3.9428
모델 체크포인트가 /content/drive/MyDrive/small_bert_korean/model_checkpoint_epoch_1.pt에 저장되었습니다.
Epoch [2/10], Loss: 0.4445, Perplexity: 1.5597
모델 체크포인트가 /content/drive/MyDrive/small_bert_korean/model_checkpoint_epoch_2.pt에 저장되었습니다.
Epoch [3/10], Loss: 0.4155, Perplexity: 1.5151
모델 체크포인트가 /content/drive/MyDrive/small_bert_korean/model_checkpoint_epoch_3.pt에 저장되었습니다.
Epoch [4/10], Loss: 0.4033, Perplexity: 1.4968
모델 체크포인트가 /content/drive/MyDrive/small_bert_korean/model_checkpoint_epoch_4.pt에 저장되었습니다.
Epoch [5/10], Loss: 0.4022, Perplexity: 1.4951
모델 체크포인트가 /content/drive/MyDrive/small_bert_korean/model_checkpoint_epoch_5.pt에 저장되었습니다.
Epoch [6/10], Loss: 0.3918, Perplexity: 1.4797
모델 체크포인트가 /content/drive/MyDrive/small_bert_korean/model_checkpoint_epoch_6.pt에 저장되었습니다.
Epoch [7/10], Loss: 0.3891, Perplexity: 1.4756
모델 체크포인트가 /content/drive/MyDrive/small_bert_korean/model_checkpoint_epoch_7.pt에 저장되었습니다.
Epoch [8/10], Loss: 0.3721, Perplexity: 1.4508
모

In [15]:
# 최종 모델 저장
final_model_path = os.path.join(SAVE_PATH, 'final_small_bert.pt')
torch.save(model.state_dict(), final_model_path)
print(f"최종 모델이 {final_model_path}에 저장되었습니다.")

# 학습 성능 지표 저장 (예시)
performance_log_path = os.path.join(SAVE_PATH, 'training_performance.txt')
with open(performance_log_path, 'w') as f:
    f.write(f"Final Average Loss: {avg_loss}\n")
    f.write(f"Final Perplexity: {math.exp(avg_loss)}\n")
    f.write(f"Total Epochs: {num_epochs}\n")
print(f"학습 성능 지표가 {performance_log_path}에 저장되었습니다.")

최종 모델이 /content/drive/MyDrive/small_bert_korean/final_small_bert.pt에 저장되었습니다.
학습 성능 지표가 /content/drive/MyDrive/small_bert_korean/training_performance.txt에 저장되었습니다.
